## Setup

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80
Wed Apr 13 13:13:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    34W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                           

In [ ]:
!pip install transformers
!pip install farasapy==0.0.14
!pip install pyarabic==0.6.14
!git clone https://github.com/aub-mind/arabert
!pip install sentencepiece==0.1.96
!pip install datasets

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 44.4 MB/s 
     |████████████████████████████████| 895 kB 42.4 MB/s 
     |████████████████████████████████| 77 kB 6.7 MB/s 
     |████████████████████████████████| 6.6 MB 44.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 126 kB 5.4 MB/s 
Cloning into 'arabert'...
remote: Enumerating objects: 564, done.
remote: Counting objects: 100% (350/350), done.
remote: Compressing objects: 100% (252/252), done.
remote: Total 564 (delta 188), reused 247 (delta 92), pack-reused 214
Receiving objects: 100% (564/564), 9.16 MiB | 23.45 MiB/s, done.
Resolving deltas: 100% (311/311), done.
     |████████████████████████████████| 1.2 MB 5.2 MB/s 
     |████████████████████████████████| 325 kB 5.2 MB/s 
     |████████████████████████████████| 136 kB 51.6 M

In [ ]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from arabert.preprocess import ArabertPreprocessor
import numpy as np
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, f1_score, precision_score,
                             recall_score)

Set up constants.


In [ ]:
DIRECT_SCORE = 0
RELATED_SCORE = 1
IRRELEVANT_SCORE = 2

## Prepare for Training

Initialize tokenizer, data collator, and AraBERT preprocessor.

In [ ]:
checkpoint = "aubmindlab/bert-base-arabertv2"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

data_collator = DataCollatorWithPadding(tokenizer)

arabert_preprocessor = ArabertPreprocessor(checkpoint)

Downloading:   0%|          | 0.00/611 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/384 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/703k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


100%|██████████| 241M/241M [00:21<00:00, 11.5MiB/s]


[2022-04-13 13:15:20,755 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


Load dataset from JSON file and split into a training and testing sets. *(You need to upload the `semeval_dataset.json` file to your Drive first, and then mount Drive to this notebook.)*

In [ ]:
semeval_dataset = load_dataset(
    'json',
    data_files={'train': '/content/drive/MyDrive/semeval_dataset_train.json',
                'dev': '/content/drive/MyDrive/semeval_dataset_dev.json',
                'test': '/content/drive/MyDrive/semeval_dataset_test.json'},
    field='data')
print(semeval_dataset.column_names)

Using custom data configuration default-cbae38a98bf01b68


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-cbae38a98bf01b68/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

{'train': ['question', 'answer', 'label'], 'dev': ['question', 'answer', 'label'], 'test': ['question', 'answer', 'label']}


Create an AraBERT preprocessing function and preprocess the dataset.

In [ ]:
def preprocess_for_arabert(example):
  example['question'] = arabert_preprocessor.preprocess(example['question'])
  example['answer'] = arabert_preprocessor.preprocess(example['answer'])
  if 'label' in example:
    if example['label'] == "direct":
      example['label'] = 0
    elif example['label'] == "related":
      example['label'] = 1
    elif example['label'] == "irrelevant":
      example['label'] = 2

  return example

preprocessed_dataset = semeval_dataset.map(preprocess_for_arabert)

Parameter 'function'=<function preprocess_for_arabert at 0x7f7b81348dd0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/2495 [00:00<?, ?ex/s]

  0%|          | 0/1000 [00:00<?, ?ex/s]

  0%|          | 0/1000 [00:00<?, ?ex/s]

Create a tokenization function and tokenize the dataset.

In [ ]:
def tokenize_function(examples):
  return tokenizer(examples['question'], examples['answer'])

tokenized_datasets = preprocessed_dataset.map(tokenize_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Check to see if it worked.

In [ ]:
print(tokenizer.decode(tokenized_datasets['train']['input_ids'][0]))

[CLS] أنا شخص لدي ل+ حي +ة خفيف +ة و+ هي غير متساوي +ة. ما ال+ حكم إذا خفف +ت +ها و+ ساوي +ت +ها على هيئ +ة ما يعرف ب+ ال+ عارض ؟ أفيدوني جزا +كم الله خير +ا. [SEP] ال+ حمد ل+ الله و+ ال+ صلا +ة و+ ال+ سلام على رسول الله و+ على آله و+ صحب +ه أما بعد : ف+ خف +ة ال+ لحي +ة و+ عدم تساوي أطراف +ها لا يبيح ل+ +ك أخذ ما دون ال+ قبض +ة من +ها ل+ حرم +ة ذلك ، إلا إذا كان +ت متباين +ة ال+ أطراف تباين +ا فاحش +ا مشو +ها ل+ ال+ خلق +ة فيقتصر على قص ال+ زائد الذي حصل ب+ +ه ال+ تشويه فقط كما هو مبين في ال+ فتوى رقم : و+ الله أعلم. [SEP]


## Training

Load AraBERT model.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were

Specify training arguments.

In [ ]:
training_args = TrainingArguments(
    "test-trainer",
    num_train_epochs= 10,
    learning_rate = 2e-5,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True, # this allows to automatically get the best model at the end based on whatever metric we want
    metric_for_best_model = 'macro_f1',
    # per_device_train_batch_size = 4, # up to 64 on 16GB with max len of 128, was 16
    # per_device_eval_batch_size = 16,
    # gradient_accumulation_steps = 15, # use this to scale batch size without needing more memory
)

In [ ]:
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  #print(classification_report(p.label_ids,preds))
  #print(confusion_matrix(p.label_ids,preds))
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  #macro_precision = precision_score(p.label_ids,preds,average='macro')
  #macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  return {       
      'macro_f1' : macro_f1,
      'accuracy': acc
  }

Start training. *(Evaluation not setup yet.)*

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['dev'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: answer, question. If answer, question are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2495
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3120


Epoch,Training Loss,Validation Loss,Macro F1,Accuracy
1,No log,0.304818,0.822536,0.879000
2,0.350800,0.328659,0.850619,0.894000
3,0.350800,0.532214,0.844111,0.891000
4,0.157400,0.598342,0.826973,0.880000
5,0.064300,0.614284,0.845343,0.892000
6,0.064300,0.619992,0.859982,0.899000
7,0.028000,0.658640,0.855608,0.898000
8,0.028000,0.683230,0.851948,0.894000
9,0.012600,0.690470,0.856381,0.897000
10,0.011600,0.698543,0.856669,0.897000


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: answer, question. If answer, question are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to test-trainer/checkpoint-312
Configuration saved in test-trainer/checkpoint-312/config.json
Model weights saved in test-trainer/checkpoint-312/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-312/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-312/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: answer, question. If answer, question are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running E

TrainOutput(global_step=3120, training_loss=0.1003816514634169, metrics={'train_runtime': 2649.4725, 'train_samples_per_second': 9.417, 'train_steps_per_second': 1.178, 'total_flos': 2943964725303924.0, 'train_loss': 0.1003816514634169, 'epoch': 10.0})

Save trained model.

In [ ]:
label_map = {
    "direct": 0,
    "related": 1,
    "irrelevant": 2
}
inv_label_map = { v:k for k, v in label_map.items() }

trainer.model.config.label2id = label_map
trainer.model.config.id2label = inv_label_map
trainer.save_model("saved_model_new")

Saving model checkpoint to saved_model_new
Configuration saved in saved_model_new/config.json
Model weights saved in saved_model_new/pytorch_model.bin
tokenizer config file saved in saved_model_new/tokenizer_config.json
Special tokens file saved in saved_model_new/special_tokens_map.json


Save model to Drive for persistance.

In [ ]:
!cp -r /content/saved_model_new /content/drive/MyDrive

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
from huggingface_hub import HfApi
api = HfApi()

# api.create_repo?

api.create_repo (
    repo_id = "saadanis", # The name of our repository, by default under your user
    private = True, # Whether the repo should be public or private
    repo_type = "model" # The type of repository, such as "model", "space", "dataset"
)

## Predict

Load saved model from Drive if it already exists.

In [ ]:
!cp -r /content/drive/MyDrive/saved_model_new /content

Load tokenizer and model from the saved model.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("saved_model_new")
model = AutoModelForSequenceClassification.from_pretrained("saved_model_new")

classes = ["direct", "related", "irrelevant"]

Didn't find file saved_model_new/added_tokens.json. We won't load it.
loading file saved_model_new/vocab.txt
loading file saved_model_new/tokenizer.json
loading file None
loading file saved_model_new/special_tokens_map.json
loading file saved_model_new/tokenizer_config.json
loading configuration file saved_model_new/config.json
Model config BertConfig {
  "_name_or_path": "saved_model_new",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "direct",
    "1": "related",
    "2": "irrelevant"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "direct": 0,
    "irrelevant": 2,
    "related": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding

Predict the relevancy of the question and answer.

In [ ]:
question= "هل يجوز الحج عن شخص متوفى شنقاً؟"
answer= "فإن كان المراد أن الشخص المتوفى شنق نفسه فهذا يعتبر منتحراً، والانتحار جريمة شنيعة توعد الله صاحبها، ولكنه إذا كان مات على الإسلام يشرع الحج عنه، بل يجب إذا كان اقتدر على الحج ولم يحج وترك مالاً يمكن أن يحج عنه به، وهذا هو مذهب الشافعية والحنابلة وهو الراجح، وقال الأحناف والمالكية لا يجب الحج عنه، وراجع للبسط في الموضوع الفتاوى ذات الأرقام التالية: ، ، ، ، ، . والله أعلم."

In [ ]:
tokenized_example = tokenizer(question, answer, return_tensors="pt")

classification_logits = model(**tokenized_example).logits

classification_results = torch.softmax(classification_logits, dim=1).tolist()[0]

for i in range(len(classes)):
    print(f"{classes[i]}: {round(classification_results[i]*100, 2)}%")


direct: 99.88%
related: 0.06%
irrelevant: 0.06%


## Test

Load saved model from Drive if it already exists.

In [ ]:
!cp -r /content/drive/MyDrive/saved_model /content

Load tokenizer and model from the saved model.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("saved_model")
model = AutoModelForSequenceClassification.from_pretrained("saved_model")

classes = ["direct", "related", "irrelevant"]

### Pointwise Testing (Outdated)

Test precision, recall, and F1 for each isolated query-answer pair.

In [ ]:
results = []

for item in tokenized_datasets['test']:
  tokenized_item = tokenizer(item['question'], item['answer'], return_tensors='pt')
  classification_logits = model(**tokenized_item).logits
  classification_results = torch.softmax(classification_logits, dim=1).tolist()[0]

  prediction_index = classification_results.index(max(classification_results))
  results.append([prediction_index, item['label']])

print(results)

[[2, 2], [1, 0], [2, 2], [1, 1], [1, 1], [1, 2], [1, 0], [1, 1], [2, 2], [2, 2], [2, 2], [2, 2], [0, 0], [2, 2], [1, 1], [0, 0], [0, 1], [2, 2], [2, 2], [2, 2], [1, 1], [2, 2], [2, 2], [2, 2], [0, 0], [2, 2], [2, 2], [2, 2], [0, 0], [1, 1], [2, 2], [2, 2], [2, 2], [1, 1], [1, 0], [1, 1], [1, 2], [0, 0], [2, 2], [2, 2], [2, 2], [1, 1], [2, 2], [0, 0], [2, 2], [2, 2], [2, 2], [1, 1], [2, 2], [0, 0], [2, 2], [2, 2], [2, 2], [0, 1], [0, 0], [2, 2], [2, 2], [2, 2], [1, 1], [0, 0], [1, 1], [2, 2], [1, 1], [2, 2], [1, 0], [2, 2], [2, 2], [2, 2], [0, 1], [0, 0], [1, 0], [2, 2], [2, 2], [2, 2], [0, 1], [2, 2], [2, 2], [0, 0], [2, 2], [1, 1], [2, 2], [0, 0], [2, 2], [2, 2], [1, 1], [2, 2], [2, 2], [1, 1], [0, 0], [2, 2], [2, 2], [1, 1], [2, 2], [2, 2], [0, 0], [2, 2], [1, 1], [2, 2], [0, 0], [2, 2], [1, 1], [2, 2], [2, 2], [1, 0], [2, 2], [1, 0], [1, 2], [1, 1], [2, 2], [2, 2], [2, 2], [0, 0], [2, 2], [1, 1], [2, 2], [2, 2], [2, 2], [0, 0], [1, 2], [1, 1], [2, 2], [1, 1], [0, 0], [2, 2], [2, 2],

In [ ]:
import json

with open("results.json", "w") as f:
  json.dump(results, f)

In [ ]:
r_pred = []
r_true = []

for result in results:
  r_pred.append(result[0])
  r_true.append(result[1])

print(r_pred)
print(r_true)

[2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 0, 2, 1, 0, 0, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 0, 1, 2, 2, 2, 1, 1, 1, 1, 0, 2, 2, 2, 1, 2, 0, 2, 2, 2, 1, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 1, 0, 1, 2, 1, 2, 1, 2, 2, 2, 0, 0, 1, 2, 2, 2, 0, 2, 2, 0, 2, 1, 2, 0, 2, 2, 1, 2, 2, 1, 0, 2, 2, 1, 2, 2, 0, 2, 1, 2, 0, 2, 1, 2, 2, 1, 2, 1, 1, 1, 2, 2, 2, 0, 2, 1, 2, 2, 2, 0, 1, 1, 2, 1, 0, 2, 2, 2, 1, 2, 1, 2, 2, 2, 0, 1, 2, 1, 0, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 0, 2, 2, 0, 2, 2, 1, 2, 0, 1, 2, 2, 1, 2, 0, 2, 2, 1, 0, 2, 2, 2, 1, 1, 2, 2, 2, 0, 2, 0, 1, 2, 2, 2, 2, 0, 1, 2, 2, 2, 1, 0, 2, 2, 2, 2, 1, 0, 2, 2, 2, 1, 0, 2, 0, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 0, 2, 0, 1, 1, 2, 1, 2, 2, 2, 0, 2, 2, 2, 1, 0, 0, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 0, 1, 1, 2, 2, 2, 0, 1, 2, 2, 2, 2, 0, 1, 2, 1, 0, 2, 2, 2, 1, 0, 2, 1, 2, 2, 2, 1, 0, 1, 1, 2, 2, 2, 2, 2, 2, 1, 0, 2, 1, 2, 2, 0, 2, 2, 2, 1, 1, 2, 1, 2, 1, 0, 2, 1, 2, 2, 0, 2, 2, 2, 0, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 1, 1, 

In [ ]:
from sklearn import metrics

print(metrics.classification_report(r_true, r_pred, digits=3))

              precision    recall  f1-score   support

           0      0.834     0.772     0.802       215
           1      0.640     0.833     0.724       222
           2      0.973     0.885     0.927       563

    accuracy                          0.849      1000
   macro avg      0.816     0.830     0.818      1000
weighted avg      0.869     0.849     0.855      1000



### Listwise Testing


In [ ]:
from sklearn.metrics import ndcg_score

In [ ]:
idcg = []
dcg = []

for i in range(int(len(tokenized_datasets['test'])/5)):
  print(f"{(i*5)+1}/{len(tokenized_datasets['test'])}")
  _idcg = []
  _dcg = []
  for j in range(5):
    item = tokenized_datasets['test'][(i*5)+j]
    _idcg.append(item['label'])
    tokenized_item = tokenizer(item['question'], item['answer'], return_tensors='pt')
    classification_logits = model(**tokenized_item).logits
    classification_results = torch.softmax(classification_logits, dim=1).tolist()[0]

    cl_index = classification_results.index(max(classification_results))
    if cl_index == 0:
      _dcg.append(DIRECT_SCORE)
    elif cl_index == 1:
      _dcg.append(RELATED_SCORE)
    elif cl_index == 2:
      _dcg.append(IRRELEVANT_SCORE)
  idcg.append(_idcg)
  dcg.append(_dcg)

print(idcg)
print(dcg)

1/1000
6/1000
11/1000
16/1000
21/1000
26/1000
31/1000
36/1000
41/1000
46/1000
51/1000
56/1000
61/1000
66/1000
71/1000
76/1000
81/1000
86/1000
91/1000
96/1000
101/1000
106/1000
111/1000
116/1000
121/1000
126/1000
131/1000
136/1000
141/1000
146/1000
151/1000
156/1000
161/1000
166/1000
171/1000
176/1000
181/1000
186/1000
191/1000
196/1000
201/1000
206/1000
211/1000
216/1000
221/1000
226/1000
231/1000
236/1000
241/1000
246/1000
251/1000
256/1000
261/1000
266/1000
271/1000
276/1000
281/1000
286/1000
291/1000
296/1000
301/1000
306/1000
311/1000
316/1000
321/1000
326/1000
331/1000
336/1000
341/1000
346/1000
351/1000
356/1000
361/1000
366/1000
371/1000
376/1000
381/1000
386/1000
391/1000
396/1000
401/1000
406/1000
411/1000
416/1000
421/1000
426/1000
431/1000
436/1000
441/1000
446/1000
451/1000
456/1000
461/1000
466/1000
471/1000
476/1000
481/1000
486/1000
491/1000
496/1000
501/1000
506/1000
511/1000
516/1000
521/1000
526/1000
531/1000
536/1000
541/1000
546/1000
551/1000
556/1000
561/1000
566/1

In [ ]:
ndcg_score(np.asarray(idcg),np.asarray(dcg))

0.9713206825807057